In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']

In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

In [16]:
index_name = "wiki"
for idx in pc.list_indexes():
    # 인덱스 이름이 "wiki"와 일치하는 경우 해당 인덱스를 삭제합니다.
    if idx.name == index_name:
        pc.delete_index(idx.name)

In [17]:
from pinecone import ServerlessSpec
pc.create_index(
    name=index_name,
    dimension=1536,  # 모델 차원
    metric="cosine",  # 모델 메트릭
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "wiki",
    "metric": "cosine",
    "host": "wiki-qkg5fg2.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1536,
    "deletion_protection": "disabled",
    "tags": null
}

In [18]:
# 'wiki' 인덱스를 가져옵니다.
index = pc.Index(index_name)

index.describe_index_stats()

c:\Users\Admin\miniconda3\envs\prompting_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

# data load

In [ ]:
# !pip install -qU langchain

In [19]:

from datasets import load_dataset
data = load_dataset("wikimedia/wikipedia", "20231101.ko", split='train[:100]')

c:\Users\Admin\miniconda3\envs\prompting_env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\datasets--wikimedia--wikipedia. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating train split: 100%|██████████| 647897/647897 [00:01<00:00, 335458.52 examples/s]


In [20]:
data[0]

{'id': '5',
 'url': 'https://ko.wikipedia.org/wiki/%EC%A7%80%EB%AF%B8%20%EC%B9%B4%ED%84%B0',
 'title': '지미 카터',
 'text': '제임스 얼 카터 주니어(, 1924년 10월 1일~)는 민주당 출신 미국의 제39대 대통령(1977년~1981년)이다.\n\n생애\n\n어린 시절 \n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n\n조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.\n\n정계 입문 \n1962년 조지아주 상원 의원 선거에서 낙선하였으나, 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사 선거에서 당선됐다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.\n\n대통령 재임 \n\n1976년 미합중국 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워서, 많은 지지를 받았는데 제럴드 포드 대통령을 누르고 당선되었다.\n\n카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.\n\n외교 정책 \n카터는 이집트와 이스라엘을 조정하여 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다. 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 그러나 1979년, 양국 간의 평화조약이 백악관에서 이루어졌다.\n\n소련과 제2차 전략 무기 제한 협상(SALT II)에 조인했다.\n\n카터는 1970년대 후반 당시 대한민국 등 인권 후진국의 국민들의 인권을 지키기 위해 노력했으

In [21]:
len(data)

100

In [ ]:
for rec in data:
    if len(rec['text']) > 35000:
        print('**')

**
**


파인콘의 경우 하나의 문장에 40kb가 넘으면 안되는데 위에 보면 두개가 넘음

In [40]:
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(model='text-embedding-3-small')

In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 400,
    chunk_overlap = 20,
    length_function = len, \
        separators = ["\n\n",'\n',' ',''] # 자르는 순서
)

splitter

In [31]:
import json
def calculate_metadata_size(metadata):
    return len(json.dumps(metadata, ensure_ascii = False).encode('utf-8'))

In [37]:
from uuid import uuid4
import time
texts = []
metas = []
count = 0
batch_size = 20

In [42]:

for i, sample in enumerate(data):
  full_text = sample['text'] #전문
  
  #메타데이터 구성
  metadata = {
    "wiki_id": str(sample['id']),
    "url": sample['url'],
    "title": sample['title'] #[:100]
  }
  
  # 스플릿터로 자르기
  chunks = splitter.split_text(full_text)
  #print(len(chunks))
  
  for i, chunk in enumerate(chunks):
    record = {
      'chunk_id':i,
      'full_text':full_text,
      **metadata
    }
    metadata_size = calculate_metadata_size(record)
    if metadata_size > 35000:
      continue
    
    texts.append(chunk)
    metas.append(record)
    count += 1
    
    # 청크 배치의 임베딩 -> 적재
    if count % batch_size == 0:
      ids = [str(uuid4()) for _ in range(len(texts))]
      embeddings = embedding.embed_documents(texts) #20개 청크의 임베딩 수행
      index.upsert(
        vectors=zip(ids, embeddings, metas),
        namespace='wiki-ns1'
      )
      texts=[]
      metas=[]
      time.sleep(1)

In [43]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'wiki-ns1': {'vector_count': 1220}},
 'total_vector_count': 1220,
 'vector_type': 'dense'}

# 질의 임베딩

In [51]:
question = ["벨기에 수도가 어디입니까?"]
emb_question = embedding.embed_documents(question)
emb_question

[[0.012805643491446972,
  -0.005858207121491432,
  0.10216851532459259,
  0.018073130398988724,
  0.012990063056349754,
  0.01929491199553013,
  -0.023790141567587852,
  0.07303020358085632,
  -0.03234260529279709,
  -0.05329729616641998,
  -0.010534975677728653,
  0.031005563214421272,
  0.006754371803253889,
  -0.02014785259962082,
  0.04232432320713997,
  0.017012717202305794,
  -0.032319553196430206,
  -0.04160969704389572,
  0.019974960014224052,
  0.04013433679938316,
  -0.020240062847733498,
  -0.004763215314596891,
  -0.011699125170707703,
  0.03778298571705818,
  0.06030524522066116,
  -0.0010568113066256046,
  0.03619236871600151,
  0.003276331350207329,
  0.01625198684632778,
  -0.03695309907197952,
  -0.012690381146967411,
  -0.028631160035729408,
  -0.02195747196674347,
  -0.04956279695034027,
  0.01379689946770668,
  -0.003212937153875828,
  0.001218899036757648,
  0.023478934541344643,
  0.006552662700414658,
  -0.026971381157636642,
  0.013554848730564117,
  -0.00677166

In [52]:
# 파이콘 질의 실헹
q_result = index.query(
    namespace = 'wiki-ns1',
    vector =  emb_question,
    top_k = 5,
    include_vector = False, 
    include_metadata = True
)
q_result

{'matches': [{'id': 'bb8d34c6-3b67-4364-a720-36d3ccf40275',
              'metadata': {'chunk_id': 2.0,
                           'full_text': '5월 31일은 그레고리력으로 151번째(윤년일 경우 152번째) '
                                        '날에 해당한다.\n'
                                        '\n'
                                        '사건 \n'
                                        ' 1933년 - 탕구 협정의 체결에 의해서 만주사변이 종결.\n'
                                        ' 1945년 - 타이베이 대공습.\n'
                                        ' 1970년 - 페루 북부에서 앙카슈 지진이 발생했다.\n'
                                        ' 2004년 - 대한민국의 무역인 김선일이 피랍됐다.\n'
                                        ' 2005년 - 마크 펠트가 자신이 딥 스로트임을 언론과의 '
                                        '인터뷰에서 밝혔다.\n'
                                        '\n'
                                        '문화 \n'
                                        ' 1935년 - 미국 캘리포니아주 로스앤젤레스에서 20세기 폭스가 '
                                        '설립되었다.\n'
                          

In [53]:
result_ids = [r.id for r in q_result.matches]
result_ids

['bb8d34c6-3b67-4364-a720-36d3ccf40275',
 'ef603ab3-0ef7-4ba0-8d93-893ccc9c89bf',
 'dd157da5-e25e-40de-8bb1-f0d3795737c1',
 '021ceb88-75fa-478a-9843-f21e1fb2226f',
 'ccec9e28-7aa1-41fc-a983-9e992c5e7872']

In [54]:
for r in q_result.matches:
    print(r.score, r.metadata) 

0.345850945 {'chunk_id': 2.0, 'full_text': '5월 31일은 그레고리력으로 151번째(윤년일 경우 152번째) 날에 해당한다.\n\n사건 \n 1933년 - 탕구 협정의 체결에 의해서 만주사변이 종결.\n 1945년 - 타이베이 대공습.\n 1970년 - 페루 북부에서 앙카슈 지진이 발생했다.\n 2004년 - 대한민국의 무역인 김선일이 피랍됐다.\n 2005년 - 마크 펠트가 자신이 딥 스로트임을 언론과의 인터뷰에서 밝혔다.\n\n문화 \n 1935년 - 미국 캘리포니아주 로스앤젤레스에서 20세기 폭스가 설립되었다.\n 1982년 - 대한민국의 첫 인터넷 개설되었다.\n 1996년\n FIFA는 2002년 축구 월드컵을 대한민국과 일본에서 공동 개최하도록 결정했다.\n 1998년 FIFA 월드컵 예선 경기에서 부르키나파소가 모리타니와 0대 0으로 비겼다.\n 2002년\n 2002년 FIFA 월드컵이 서울 월드컵 경기장에서 개막했다. 그리고 여기서 치러진 개막전에서 처녀 출전국인 세네갈이 전 대회 우승국인 프랑스를 1:0으로 꺾는 파란을 일으켰다.\n 대한민국에서 제주방송(JIBS) 텔레비전이 개국하다.\n 2017년\n 수도권에서 지상파 UHD 방송이 개시되었다.\n 서울메트로와 서울특별시 도시철도공사가 서울교통공사로 통합 출범하였다.\n 2023년 - 수도권제2순환고속도로 조안 나들목 ~ 양평 나들목 구간이 개통되었다.\n\n탄생 \n 1162년 - 칭기즈 칸, 몽골 제국 초대 대칸. (~1227년)\n 1469년 - 포르투갈왕 마누엘 1세. (~1521년)\n 1819년 - 미국의 시인 겸 수필가 월트 휘트먼. (~1892년)\n 1835년 - 일본 막부 말기 신센구미 부장 히지카타 도시조. (~1869년)\n 1857년 - 교황 비오 11세, 259대 로마 교황. (~1939년)\n 1887년 - 프랑스의 외교관 겸 시인 생존 페르스. (~1975년)\n 1892년 - 나치 독일의 정치가 그레고어 슈트라서. (~1934

# 랭체인의 api로 결과 비교하기

In [55]:
question

['벨기에 수도가 어디입니까?']

In [60]:
from langchain_pinecone import PineconeVectorStore

# VectorStore 초기화
vector_store = PineconeVectorStore(
    index=index,
    embedding=embedding,
    text_key='full_text'
)

# 질의 실행
docs = vector_store.similarity_search(
    query=question[0],  # question이 ["텍스트"] 형태 리스트라면 question[0] 사용
    k=5,
    namespace='wiki-ns1'
)

docs


c:\Users\Admin\miniconda3\envs\prompting_env\Lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


[Document(id='bb8d34c6-3b67-4364-a720-36d3ccf40275', metadata={'chunk_id': 2.0, 'title': '5월 31일', 'url': 'https://ko.wikipedia.org/wiki/5%EC%9B%94%2031%EC%9D%BC', 'wiki_id': '41'}, page_content='5월 31일은 그레고리력으로 151번째(윤년일 경우 152번째) 날에 해당한다.\n\n사건 \n 1933년 - 탕구 협정의 체결에 의해서 만주사변이 종결.\n 1945년 - 타이베이 대공습.\n 1970년 - 페루 북부에서 앙카슈 지진이 발생했다.\n 2004년 - 대한민국의 무역인 김선일이 피랍됐다.\n 2005년 - 마크 펠트가 자신이 딥 스로트임을 언론과의 인터뷰에서 밝혔다.\n\n문화 \n 1935년 - 미국 캘리포니아주 로스앤젤레스에서 20세기 폭스가 설립되었다.\n 1982년 - 대한민국의 첫 인터넷 개설되었다.\n 1996년\n FIFA는 2002년 축구 월드컵을 대한민국과 일본에서 공동 개최하도록 결정했다.\n 1998년 FIFA 월드컵 예선 경기에서 부르키나파소가 모리타니와 0대 0으로 비겼다.\n 2002년\n 2002년 FIFA 월드컵이 서울 월드컵 경기장에서 개막했다. 그리고 여기서 치러진 개막전에서 처녀 출전국인 세네갈이 전 대회 우승국인 프랑스를 1:0으로 꺾는 파란을 일으켰다.\n 대한민국에서 제주방송(JIBS) 텔레비전이 개국하다.\n 2017년\n 수도권에서 지상파 UHD 방송이 개시되었다.\n 서울메트로와 서울특별시 도시철도공사가 서울교통공사로 통합 출범하였다.\n 2023년 - 수도권제2순환고속도로 조안 나들목 ~ 양평 나들목 구간이 개통되었다.\n\n탄생 \n 1162년 - 칭기즈 칸, 몽골 제국 초대 대칸. (~1227년)\n 1469년 - 포르투갈왕 마누엘 1세. (~1521년)\n 1819년 - 미국의 시인 겸 수필가 월트 휘트먼. (~1892년)\n 1835년 - 일본 막부

# Augmentation

In [62]:
country = '벨기에'
query = f"{country}의 여행을 위한 관광지와 문화"

docs = vector_store.similarity_search(
    query=query,
    k=5,
    namespace='wiki-ns1'
)

In [63]:
context = ""
for result in docs :
    context += result.page_content

In [64]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = 'gpt-4o-mini')

In [66]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages([
    ('system', '당신은 베테랑 여행 가이드 입니다, 고객 맞춤 여행 일정 수립을 도와드립니다. 제한된 정보라도 최대한 활용해서 도움을 주세요'),
    ('human', '저는{country}로 여행을 계획중입니다. 아래 context에서 {country}에 관련된 정보를 찾아서 여행 조언을 해주세요. 만약 직덥적인 정보가 없다면, 수도나 관련된 정보를 활용해 주세요 context :{context}'),
])

chat_template

ChatPromptTemplate(input_variables=['context', 'country'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 베테랑 여행 가이드 입니다, 고객 맞춤 여행 일정 수립을 도와드립니다. 제한된 정보라도 최대한 활용해서 도움을 주세요'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'country'], input_types={}, partial_variables={}, template='저는{country}로 여행을 계획중입니다. 아래 context에서 {country}에 관련된 정보를 찾아서 여행 조언을 해주세요. 만약 직덥적인 정보가 없다면, 수도나 관련된 정보를 활용해 주세요 context :{context}'), additional_kwargs={})])

In [67]:
prompt = chat_template.format_messages(country = country, context = context)
print(prompt)

[SystemMessage(content='당신은 베테랑 여행 가이드 입니다, 고객 맞춤 여행 일정 수립을 도와드립니다. 제한된 정보라도 최대한 활용해서 도움을 주세요', additional_kwargs={}, response_metadata={}), HumanMessage(content='저는벨기에로 여행을 계획중입니다. 아래 context에서 벨기에에 관련된 정보를 찾아서 여행 조언을 해주세요. 만약 직덥적인 정보가 없다면, 수도나 관련된 정보를 활용해 주세요 context :아쿠타가와 류노스케(, 1892년 3월 1일~1927년 7월 24일)는 일본의 소설가이다. 호는 징강당주인(澄江堂主人)이며 하이쿠 작가로서의 호는 가키(我鬼)이다.\n\n그의 작품은 대부분이 단편 소설이다. 「참마죽」, 「덤불 속」, 「지옥변」 등 주로 일본의 《곤자쿠 이야기집》·《우지슈이 이야기》 등 전통적인 고전들에서 제재를 취하였다. 또한 「거미줄(원제: 蜘蛛の糸)」, 「두자춘(杜子春)」 등 어린이를 위한 작품도 남겼으며, 예수를 학대한 유대인이 예수가 세상에 다시 올 때까지 방황한다는 상상력을 발휘한 「방황하는 유대인」도 있다.\n\n생애\n\n유년 시절 \n1892년(메이지 25년) 3월 1일 도쿄에서 우유 판매업자였던 아버지 니하라 도시조(新原敏三)와 어머니 후쿠(フク) 사이의 아들로 태어났다(아쿠타가와라는 성은 원래 그의 어머니쪽 성씨였다). 이때 태어난 시간이 공교롭게도 진년(辰年) 진월(辰月) 진일(辰日) 진시(辰時)였기 때문에 \'용(龍)\' 자를 이름에 넣어 류노스케(龍之介)라 짓게 되었다고 전하나, 실제 그가 태어난 1892년 3월 1일은 간지로는 임진년·임인월·임진일에 해당하며, 출생 시각에 대해서는 자료가 없기 때문에 확실한 것이 없다. 이름도 호적상으로는 \'龍之介\'이지만 그가 양자로 들어갔던 아쿠타가와 집안이나 졸업한 학교의 명단 등의 문서에는 \'龍之助\'로 되어 있다(아쿠타가와 자신은 \'龍之助\' 표기를 싫어했다).\n\n가족 \n원래 류노스케의 위

In [68]:
response = llm.invoke(prompt)
response

AIMessage(content='벨기에 여행을 계획하고 계신다니 멋진 선택입니다! 수도 브뤼셀뿐만 아니라, 벨기에는 다양한 매력을 지닌 도시들과 아름다운 풍경이 가득합니다. 아래는 기본적인 여행 일정과 벨기에의 주요 관광 명소에 대한 추천입니다.\n\n### 기본 여행 일정 (7일 기준)\n\n#### 1일차: 브뤼셀\n- **오전**: **그랑플라스** 방문 – 중세 건축물과 화려한 마을 광장.\n- **오후**: **브뤼셀 왕궁**과 **네덜란드 대사관 건물** 탐방.\n- **저녁**: **맥주 시음** – 벨기에 맥주 전문 바에서 다양한 맥주를 맛보세요.\n\n#### 2일차: 브뤼셀 & 브뤼지\n- **오전**: 브뤼셀에서 **아토미움** 방문 – 현대적인 풍경을 감상하세요.\n- **오후**: **브뤼지**로 이동 (기차 약 1시간) – **브뤼지 운하**와 **성혈 성당** 탐방.\n- **저녁**: 운하를 따라 산책하며 현지 식사.\n\n#### 3일차: 브뤼지\n- **오전**: **마르케 델 페어 동상** 및 **브뤼지 구시가지** 탐방.\n- **오후**: **반 고흐 미술관** 또는 **질리제 미술관** 방문.\n- **저녁**: 브뤼지의 매력적인 레스토랑에서 저녁.\n\n#### 4일차: 겐트\n- **오전**: **겐트**로 이동 (기차 약 30분) – **겐트 성**과 **세인트 바바 성당** 방문.\n- **오후**: **그라스리 플라츠** 탐방 – 벨기에의 식당과 카페에 앉아 식사.\n- **저녁**: **겐트에서 플라멩코 공연 관람**.\n\n#### 5일차: 앤트워프\n- **오전**: **앤트워프**로 이동 (기차 약 1시간) – **앤트워프 대성당** 및 **다이아몬드 지구** 탐방.\n- **오후**: **피카소 미술관** 또는 **모네의 풍경화** 보기.\n- **저녁**: 앤트워프의 전통 레스토랑에서 저녁.\n\n#### 6일차: 다남지구 & 루벤\n- **오전**: **다남지구** 드라이브 및 탐방 – 독특한 경치

In [69]:
response.content

'벨기에 여행을 계획하고 계신다니 멋진 선택입니다! 수도 브뤼셀뿐만 아니라, 벨기에는 다양한 매력을 지닌 도시들과 아름다운 풍경이 가득합니다. 아래는 기본적인 여행 일정과 벨기에의 주요 관광 명소에 대한 추천입니다.\n\n### 기본 여행 일정 (7일 기준)\n\n#### 1일차: 브뤼셀\n- **오전**: **그랑플라스** 방문 – 중세 건축물과 화려한 마을 광장.\n- **오후**: **브뤼셀 왕궁**과 **네덜란드 대사관 건물** 탐방.\n- **저녁**: **맥주 시음** – 벨기에 맥주 전문 바에서 다양한 맥주를 맛보세요.\n\n#### 2일차: 브뤼셀 & 브뤼지\n- **오전**: 브뤼셀에서 **아토미움** 방문 – 현대적인 풍경을 감상하세요.\n- **오후**: **브뤼지**로 이동 (기차 약 1시간) – **브뤼지 운하**와 **성혈 성당** 탐방.\n- **저녁**: 운하를 따라 산책하며 현지 식사.\n\n#### 3일차: 브뤼지\n- **오전**: **마르케 델 페어 동상** 및 **브뤼지 구시가지** 탐방.\n- **오후**: **반 고흐 미술관** 또는 **질리제 미술관** 방문.\n- **저녁**: 브뤼지의 매력적인 레스토랑에서 저녁.\n\n#### 4일차: 겐트\n- **오전**: **겐트**로 이동 (기차 약 30분) – **겐트 성**과 **세인트 바바 성당** 방문.\n- **오후**: **그라스리 플라츠** 탐방 – 벨기에의 식당과 카페에 앉아 식사.\n- **저녁**: **겐트에서 플라멩코 공연 관람**.\n\n#### 5일차: 앤트워프\n- **오전**: **앤트워프**로 이동 (기차 약 1시간) – **앤트워프 대성당** 및 **다이아몬드 지구** 탐방.\n- **오후**: **피카소 미술관** 또는 **모네의 풍경화** 보기.\n- **저녁**: 앤트워프의 전통 레스토랑에서 저녁.\n\n#### 6일차: 다남지구 & 루벤\n- **오전**: **다남지구** 드라이브 및 탐방 – 독특한 경치를 사진에 담아보세요.\n- **